In [1]:
import requests
import pandas as pd
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
# Set my players
teamId = 2
response = requests.get('https://fantasy.espn.com/apis/v3/games/flb/seasons/2020/segments/0/leagues/12442?rosterForTeamId={}&view=mRoster'.format(teamId), cookies = {"swid":"{282E6323-08FA-41BD-AC92-7D5667C61C49}", "espn_s2":"AECzXv%2BX6EDNinqK4HVwYewHkXaEHrvK2AFo9%2FOaZNA2qaUBmNCye3zJC6AI1hojKXycBBaCu0hUD7FkrxUHt84Buvf8cvWntOy22kbNWJtrfgX716wUgCRS5xeg6RntVhAU31D%2FxHoFfIkqB17mZ7QNtvu%2BiQsd54pYmi%2FBKdL%2B9Ct7t3SH2bsOlSW5RgeLlPx5NgLX6P9wwnaTqxYRt6qzZ2P17%2B4nlhvEW126C71dq2AGYg2sTXefgiJ7UfiR7%2Bw8lb3Vi%2Fa5q54M331GR9NbkCAJyy4bgvLqQftvIafm2A%3D%3D"})

teams = response.json()["teams"]
my_team = []
for team in teams:
    if 'roster' in team.keys():
        my_team = team

players = my_team["roster"]["entries"]

my_players = []
for player in players:
    my_players.append(player["playerPoolEntry"]["player"]["fullName"])
my_players

['Joey Gallo',
 'Justin Verlander',
 'Jacob deGrom',
 'Patrick Corbin',
 'Whit Merrifield',
 'Marcus Semien',
 'Jose Abreu',
 'Jeff McNeil',
 'Tim Anderson',
 'Raisel Iglesias',
 'Archie Bradley',
 'Willson Contreras',
 'J.D. Davis',
 'Mark Melancon',
 'Dansby Swanson',
 'Kyle Seager',
 'Jose Urquidy',
 'Robinson Cano',
 'Brad Keller',
 'Randal Grichuk',
 'Drew Pomeranz',
 'Jeremy Jeffress',
 'Brad Miller',
 'Rafael Dolis',
 'Tyler Mahle']

In [3]:
# Scrape data
for i in range(1,31):
    url = "https://www.fangraphs.com/dailyprojections.aspx?pos=all&stats=bat&type=sabersim&team={}".format(i)
    if i == 1:
        df = pd.read_html(url)[8]
    else:
        temp_df = pd.read_html(url)[8]
        df = df.append(temp_df, ignore_index = True)

In [4]:
# Filter data to only my players
my_players_df = df[df['Name'].isin(my_players)]

In [5]:
# Calculate a new column called 'Score' as the average of the three daily fantasy projections
scores = ((my_players_df['DraftKings'] + my_players_df['FanDuel'] + my_players_df['Yahoo']) / 3).astype(float)
my_players_df.insert(len(my_players_df.columns), 'Score', scores.round(2))

In [6]:
# Now drop the three daily fantasy projections
my_players_df_dropped = my_players_df.drop(['DraftKings','FanDuel','Yahoo'], axis = 1)

In [7]:
# Sort dataframe by 'Score' (descending) and display results
my_players_df_sorted = my_players_df_dropped.sort_values(by = 'Score', ascending = False)

In [8]:
# Remove index
my_players_df_final = my_players_df_sorted.reset_index(drop = True)
my_players_df_final

,Name,Team,Game,Pos,PA,H,1B,2B,3B,HR,R,RBI,SB,CS,BB,SO,Score
0,Tim Anderson,White Sox,DET @ CWS,SS,4.76,1.46,0.97,0.29,0.01,0.19,0.77,0.66,0.23,0.07,0.23,0.98,11.65
1,Jeff McNeil,Mets,NYM @ TOR,3B,5.01,1.28,0.79,0.3,0.02,0.18,0.77,0.64,0.1,0.05,0.6,0.88,11.38
2,Jose Abreu,White Sox,DET @ CWS,1B,4.4,1.24,0.7,0.27,0,0.27,0.74,0.86,0.02,0.01,0.37,0.78,11.29
3,J.D. Davis,Mets,NYM @ TOR,LF,4.89,1.1,0.66,0.23,0,0.21,0.78,0.67,0.04,0.02,0.75,1.31,11.01
4,Whit Merrifield,Royals,PIT @ KC,2B,4.63,1.29,0.86,0.28,0.04,0.1,0.62,0.55,0.23,0.09,0.32,0.59,10.25
5,Willson Contreras,Cubs,CHC @ MIL,C,4.68,0.94,0.56,0.19,0.01,0.18,0.58,0.59,0.05,0.03,0.62,1.46,9.27
6,Robinson Cano,Mets,NYM @ TOR,2B,4.47,1.15,0.72,0.28,0,0.15,0.6,0.67,0.01,0.01,0.38,0.65,9.27
7,Brad Miller,Cardinals,CIN @ STL,2B,4.39,0.87,0.52,0.19,0.03,0.13,0.54,0.62,0.05,0.03,0.71,1.11,9.07
8,Dansby Swanson,Braves,ATL @ WSH,SS,4.5,1.01,0.64,0.24,0.01,0.12,0.55,0.57,0.09,0.04,0.49,1.24,8.94
9,Kyle Seager,Mariners,SEA @ ARI,3B,4.43,0.95,0.54,0.25,0.02,0.15,0.48,0.54,0.03,0.02,0.54,0.76,8.60


In [9]:
# Define function to turn dataframe into visually appealling table in email
def write_to_html_file(df, title='', filename=''):
    '''
    Write an entire dataframe to an HTML file with nice formatting.
    '''

    result = '''
<html>
<head>
<style>

    h2 {
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
    }
    table { 
        margin-left: auto;
        margin-right: auto;
    }
    table, th, td {
        border: 1px solid black;
        border-collapse: collapse;
    }
    th, td {
        padding: 5px;
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    table tbody tr:hover {
        background-color: #dddddd;
    }
    .wide {
        width: 90%; 
    }

</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    result += df.to_html(classes='wide', escape=False)
    result += '''
</body>
</html>
'''
    if filename == '':
        return result
    with open(filename, 'w') as f:
        f.write(result)

In [10]:
sender_email = "peteb206@gmail.com"
receiver_email = "peteb206@gmail.com"

message = MIMEMultipart("alternative")
message["Subject"] = "Projections for my Fantasy Baseball Team Today per SaberSim"
message["From"] = sender_email
message["To"] = receiver_email

# Create the HTML version of your message
html = write_to_html_file(my_players_df_final, 'Daily Projections')

# Turn this into a html MIMEText object
part1 = MIMEText(html, "html")

# Add HTML parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(part1)

smtp_server = "smtp.gmail.com"
port = 587  # For starttls
password = "Spartans_1"

# Create a secure SSL context
context = ssl.create_default_context()

# Try to log in to server and send email
try:
    server = smtplib.SMTP(smtp_server,port)
    server.ehlo() # Can be omitted
    server.starttls(context=context) # Secure the connection
    server.ehlo() # Can be omitted
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )
except Exception as e:
    # Print any error messages to stdout
    print(e)
finally:
    server.quit()